# Interacting with OpenAI API using Python

1. Install required packages

```bash
pip install openai python-dotenv
```
* openai: Official OpenAI Python client library
* python-dotenv: Library to load environment variables from a `.env` file

1. Create a `.env` file in the project root directory

OPENAI_API_KEY=your_openai_api_key_here

* Note: Never commit your `.env` file to version control to keep your API key secure. (add to .gitignore)

In [3]:
# Instantiation the client

from openai import OpenAI
import os
from dotenv import load_dotenv

# Load .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
# Basic chat completion

def chat_with_gpt(prompt, model="gpt-4o-mini", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()  # .content is now a property, not dict key

# Test
chat_with_gpt("Hello, how are you?", temperature=0)

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

In [ ]:
# Conversation with history (stateful chat)

class Chatsession:
    def __init__(self, system_prompt="You are a helpful assistant.", model="gpt-4o-mini", temperature=0.7):
        self.messages = [{"role": "system", "content": system_prompt}]
        self.model = model
        self.temperature = temperature

    def send_message(self, user_message):
        self.messages.append({"role": "user", "content": user_message})
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
        )
        assistant_message = response.choices[0].message.content.strip()  # .content is now a property, not dict key
        self.messages.append({"role": "assistant", "content": assistant_message})
        return assistant_message

# Usages
session = Chatsession("You are ai coding assistant", "gpt-4o-mini", 0.7)
print(session.send_message("how to add a file to gitignore?"))


In [6]:
import time
import openai

def robust_chat(prompt, max_retries=3, model="gpt-4o-mini", temperature=0.7):
    for attempt in range(max_retries):
        try:
            response = chat_with_gpt(prompt, model, temperature)
            return response
        except openai.error.RateLimitError as e:
            print(f"Rate limit hit. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)
        except openai.error.AuthenticationError:
            raise ValueError("Invalid API key. Check .env.")
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

    print("Max retries exceeded.")
    return None
# Usage

print(robust_chat("Hello, Tell me a joke!", temperature=2.0))

Certainly! Here’s one for you:

Why did the scarecrow win an award? 

Because he was outstanding in his field!


In [7]:
fact_sheet = f"""
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [8]:
# Applying the function

role = "AI Coding Assistant" # or "AI Product Review Assistant", ...
command = "extract" # or "summarize", 

prompt = f"""
You are {role}. 
Your task is to generate a short {command} of a product review from an ecommerce site to give feedback to the Shipping deparmtment. 

{command} the review below, delimited by triple backticks, in at most 30 words, and focusing on any aspects that mention usage of the product.
Review:```{fact_sheet}```
"""
response = robust_chat(prompt)
print(response)

The chair is suitable for both home and business settings, offering comfort with adjustable height and various upholstery options for versatile usage.


In [9]:
from IPython.display import display, HTML
display(HTML(response))

In [ ]:
# Interactive Chatbot with UI
import ipywidgets as widgets
from IPython.display import display, clear_output

# Ensure Chatsession is available; if not, provide a minimal fallback that uses chat_with_gpt
try:
    _ = Chatsession
except NameError:
    try:
        _ = chat_with_gpt  # check if chat_with_gpt exists
        class Chatsession:
            def __init__(self, system_prompt="You are a helpful assistant.", model="gpt-4o-mini", temperature=0.7):
                self.system_prompt = system_prompt
                self.model = model
                self.temperature = temperature

            def send_message(self, user_message):
                prompt = f"{self.system_prompt}\nUser: {user_message}"
                return chat_with_gpt(prompt, model=self.model, temperature=self.temperature)
    except NameError:
        raise NameError("Chatsession is not defined and chat_with_gpt is unavailable. Run the cells that define Chatsession or chat_with_gpt first.")

class InteractiveChatbot:
    def __init__(self):
        self.session = Chatsession("You are a helpful assistant.")
        self.output = widgets.Output()
        self.text_input = widgets.Text(placeholder="Ask me anything...")
        self.send_button = widgets.Button(description="Send")
        
        self.send_button.on_click(self.send_message)
        self.text_input.on_submit(self.send_message)
        
    def send_message(self, b):
        if self.text_input.value.strip():
            with self.output:
                print(f"You: {self.text_input.value}")
                response = self.session.send_message(self.text_input.value)
                print(f"Bot: {response}\n")
            self.text_input.value = ""
    
    def display(self):
        display(widgets.VBox([self.output, widgets.HBox([self.text_input, self.send_button])]))

# Create and display chatbot
chatbot = InteractiveChatbot()
chatbot.display()